In [14]:
from utility import MongoDBHandler
import pandas as pd
from pymongo import UpdateOne

In [15]:
def remove_suffixes(df, *args):
    for i in args:
        df = df.rename(columns={col: col.replace(i,'') for col in df.columns})
    return df

def add_suffixes(df, suffix):
    return df.rename(columns={col: col+suffix for col in df.columns})

In [16]:
mongo_handler = MongoDBHandler('mongodb://localhost:27017', 'ecommerce')
mongo_handler.connect()

In [10]:
df_product = pd.DataFrame(list(mongo_handler.find('stg_amz__product_details', {})))
df_bs = pd.DataFrame(list(mongo_handler.find('trf_amz__best_sellers', {'isLatest':True})))

In [13]:
pd.merge(df_bs, df_product,on='asin', how='inner')[['asin']].drop_duplicates()

,asin
0,B0BK1457X3
2,B07XTJDJHN
4,B07PK41FL4
6,B0C7BM18PB
8,B01GRJGM0Y
...,...
4578,B08WPWTZN4
4580,B0D2V7MY5Y
4582,B07RDX8MLH
4650,B0CXXLRQRW


In [20]:
df_trf = pd.DataFrame(list(mongo_handler.find('trf_amz__best_sellers', {'isLatest': True})))
df_trf['d_rank'] = df_trf.groupby(['category','subCategory','rank'])['loadTimestamp'].rank(method='dense', ascending=True)
df_upsert = df_trf[df_trf['d_rank']>1][['_id', 'isLatest']].assign(isLatest=False)
df_upsert

,_id,isLatest


In [19]:
if not df_upsert.empty:
    mongo_handler.bulk_upsert(
        'trf_amz__best_sellers', 
        df_upsert, 
        filter_cols=['_id'],
        upsert=True
    )
else:
    print("NO DATA TO UPSERT")
    